In [1]:
import ast

import pandas as pd

# Dataset manipulation
### extract user info and review text from raw recipe dataset

In [2]:
interactions = pd.read_csv("../data/foodRecSys-V1_kaggle/raw-data_recipe.csv") # recipes raw

In [3]:
cols_to_keep = ["recipe_id", "aver_rate", "review_nums", "reviews"]

In [4]:
interactions = interactions[cols_to_keep]
interactions

,recipe_id,aver_rate,review_nums,reviews
0,222388,5.000000,3,"{8542392: {'rating': 5, 'followersCount': 11, ..."
1,240488,4.764706,29,"{3574785: {'rating': 5, 'followersCount': 0, '..."
2,218939,4.571429,12,"{13774946: {'rating': 5, 'followersCount': 0, ..."
3,87211,4.625000,163,"{1563136: {'rating': 5, 'followersCount': 0, '..."
4,245714,4.500000,2,"{2945555: {'rating': 5, 'followersCount': 6690..."
...,...,...,...,...
49693,222886,3.500000,4,"{2043209: {'rating': 5, 'followersCount': 1509..."
49694,25650,4.333333,3,"{1380296: {'rating': 4, 'followersCount': 0, '..."
49695,23544,3.615385,21,"{6412992: {'rating': 4, 'followersCount': 7, '..."
49696,170710,4.714286,19,"{2381248: {'rating': 5, 'followersCount': 2, '..."


In [5]:
interactions["reviews"] = interactions.reviews.apply(ast.literal_eval)

In [6]:
type(interactions.reviews[0])

dict

In [7]:
# black magic for 
# new row for each item (review) with key (user_id) and value (user_info) in separate columns

interactions = (
    interactions
        .assign(reviews=interactions.reviews.map(lambda d: d.items()))
        .explode("reviews")
        .assign(
            user_id=lambda df: df.reviews.str.get(0),
            user_info=lambda df: df.reviews.str.get(1)
        )
        .drop(columns="reviews")
        .reset_index(drop=True)
)

In [8]:
interactions.head(3)

,recipe_id,aver_rate,review_nums,user_id,user_info
0,222388,5.0,3,8542392,"{'rating': 5, 'followersCount': 11, 'madeRecip..."
1,222388,5.0,3,11174581,"{'rating': 5, 'followersCount': 8, 'madeRecipe..."
2,222388,5.0,3,8262477,"{'rating': 5, 'followersCount': 0, 'madeRecipe..."


In [9]:
interactions.shape

(3794003, 5)

In [10]:
interactions.user_info[0]

{'rating': 5,
 'followersCount': 11,
 'madeRecipesCount': 18,
 'favoritesCount': 200,
 'dateLastModified': '2017-04-22T12:46:43.663',
 'text': "Best breakfast ever! I ran out of paprika while seasoning, so I used garlic piercer on the other half of the batch. Very good! Can't wait to make it again... and figure out how to use the drippings!",
 'followingCount': 0}

In [57]:
interactions["user_info"][0:5].apply(pd.Series)

,rating,followersCount,madeRecipesCount,favoritesCount,dateLastModified,text,followingCount
0,5,11,18,200,2017-04-22T12:46:43.663,Best breakfast ever! I ran out of paprika whil...,0
1,5,8,55,101,2013-06-20T15:50:25.96,Awesome!\nIt's amazing.,0
2,5,0,1,52,2015-02-14T07:27:51.307,The flavors came together well and it really w...,0
3,5,0,4,118,2017-10-07T18:20:08.973,"Like most, I changed it a bit. Not a fan of T...",0
4,2,0,83,170,2018-01-06T00:06:09.563,This was one of the worst recipes I have ever ...,0


In [65]:
# expand user_info to columns

# interactions = pd.concat([interactions[["recipe_id", "aver_rate", "review_nums"]], interactions["user_info"].apply(pd.Series)], axis=1)

In [49]:
i = 0
chunksize = 100_000
header = True

while i < interactions.shape[0]:
    chunk = interactions[i : i + chunksize]
    chunk = pd.concat([chunk[["recipe_id", "user_id"]], chunk["user_info"].apply(pd.Series)], axis=1)
    chunk.text = chunk.text.str.replace("\n", " ").str.replace("\r", " ")

    chunk.to_csv("../data/raw-data_interactions_verbose.csv", index = False, header = header, mode='a')
    
    i += chunksize
    header = False


In [45]:
print(chunk.text[chunk.user_id == 9470683].values)

['Wings came out great! A few changes: 1. Cut up the rosemary into smaller pieces and sprinkled it over all the wings. 2. Opened up the garlic and sprinkled it over all the wings. \r \r Then when I added the spice mixture the rosemary and garlic mixed into the wings more. I believe this gave the dish more flavor. I baked them 20 min on one side, took them out and turned them then baked another 20 minutes on the other side. \r \r Instead of broiling them, I grilled them one by one on the grill. They came out really well. I think the grilling really sealed in the flavor. \r \r Thanks for the recipe!']


In [50]:
interactions_verbose = pd.read_csv("../data/raw-data_interactions_verbose.csv", usecols = ["user_id", "recipe_id", "dateLastModified"])

In [54]:
interactions_verbose = interactions_verbose.drop_duplicates()

In [58]:
interactions_verbose

,recipe_id,user_id,dateLastModified
0,222388,8542392,2017-04-22T12:46:43.663
1,222388,11174581,2013-06-20T15:50:25.96
2,222388,8262477,2015-02-14T07:27:51.307
3,240488,3574785,2017-10-07T18:20:08.973
4,240488,12145410,2018-01-06T00:06:09.563
...,...,...,...
3794035,170710,12318233,2013-11-03T11:09:47.763
3794036,170710,51226,2009-07-18T19:33:36.317
3794037,170710,3616571,2011-09-20T10:45:59.587
3794038,79774,1245064,2006-01-21T20:57:09.327


In [61]:
interactions_raw = pd.read_csv("../data/foodRecSys-V1_kaggle/raw-data_interaction.csv")

In [62]:
interactions_raw

,user_id,recipe_id,rating,dateLastModified
0,8542392,222388,5,2017-04-22T12:46:43.663\n
1,11174581,222388,5,2013-06-20T15:50:25.96\n
2,8262477,222388,5,2015-02-14T07:27:51.307\n
3,3574785,240488,5,2017-10-07T18:20:08.973\n
4,12145410,240488,2,2018-01-06T00:06:09.563\n
...,...,...,...,...
3793998,12318233,170710,5,2013-11-03T11:09:47.763\n
3793999,51226,170710,5,2009-07-18T19:33:36.317\n
3794000,1476783,170710,5,2009-07-27T10:21:45.823\n
3794001,1245064,79774,4,2006-01-21T20:57:09.327\n
